In [ ]:
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from skimage import io, color, transform, feature
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Path to the folders containing real and fake images
original_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
synthetic_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [ ]:
# Function to load and preprocess images
def load_and_preprocess_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            images.append(img)
    return images

In [ ]:
fake_images = load_and_preprocess_images(synthetic_folder_path)
real_images = load_and_preprocess_images(original_folder_path)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels = np.zeros(len(real_images))
fake_labels = np.ones(len(fake_images))

In [ ]:
# Combine real and fake data
all_images = np.concatenate((real_images, fake_images), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []

    for image in images:
        keypoints, descriptors = sift.detectAndCompute(image, None)
        keypoints_list.append(keypoints)
        descriptors_list.append(descriptors)

    return keypoints_list, descriptors_list

# Extract SIFT features from all images
all_keypoints, all_descriptors = extract_sift_features(all_images)

In [ ]:
def keypoints_to_vectors(keypoints_list, descriptors_list):
    sift_vector_length = 128  # SIFT descriptor length

    features_vectors = []
    for descriptors in descriptors_list:
        if descriptors is None:
            # If no keypoints are detected, add zeros as a placeholder
            features_vectors.append(np.zeros(sift_vector_length))
        else:
            # Randomly choose one descriptor from multiple descriptors
            random_idx = np.random.randint(0, descriptors.shape[0])
            features_vectors.append(descriptors[random_idx])

    return np.array(features_vectors)

# Convert keypoints and descriptors to fixed-length vectors
all_features = np.array(keypoints_to_vectors(all_keypoints, all_descriptors))

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

In [ ]:
# Define your neural network architecture
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
feature_dim = all_features.shape[1]
model = build_model((feature_dim,))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
750/750 [==============================] - 12s 10ms/step - loss: 0.5337 - accuracy: 0.7810 - val_loss: 0.3631 - val_accuracy: 0.8827
Epoch 2/10
750/750 [==============================] - 6s 7ms/step - loss: 0.4020 - accuracy: 0.8739 - val_loss: 0.3633 - val_accuracy: 0.8827
Epoch 3/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3889 - accuracy: 0.8769 - val_loss: 0.3637 - val_accuracy: 0.8827
Epoch 4/10
750/750 [==============================] - 6s 9ms/step - loss: 0.3810 - accuracy: 0.8770 - val_loss: 0.3599 - val_accuracy: 0.8827
Epoch 5/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3776 - accuracy: 0.8770 - val_loss: 0.3616 - val_accuracy: 0.8827
Epoch 6/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3766 - accuracy: 0.8770 - val_loss: 0.3599 - val_accuracy: 0.8827
Epoch 7/10
750/750 [==============================] - 6s 7ms/step - loss: 0.3745 - accuracy: 0.8770 - val_loss: 0.3586 - val_accuracy: 0.8827
Epoc

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

188/188 [==============================] - 1s 3ms/step - loss: 0.3602 - accuracy: 0.8827
Test accuracy: 0.8827160596847534
